In [2]:
from drowsiness_detection import config

config.set_paths(30, 10)
import numpy as np
import random
import matplotlib.pyplot as plt

from drowsiness_detection.data import load_experiment_search_results

plt.rcParams["figure.figsize"] = 14, 7

In [2]:
# fixed experiment params
# EXPERIMENT_IDS = (31, 32)
EXPERIMENT_IDS = (27,)


In [3]:
search_results = load_experiment_search_results(experiment_id=27)

In [4]:
search_results.estimator

Pipeline(steps=[('scaler',
                 <drowsiness_detection.models.ThreeDStandardScaler object at 0x7f21aa669a00>),
                ('classifier',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f21aa669a60>)])

In [8]:
from drowsiness_detection.data import load_nn_data, train_test_split_by_subjects




X_train, X_val, X_test, y_train, y_val, y_test = load_preprocessed_train_test_splits_nn(data_path=config.PATHS.WINDOW_DATA, exclude_sess_type="a",num_targets=2,seed=45, test_size=0.2)


In [9]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")


X_train shape: (20913, 300, 23), y_train shape: (20913,)
X_test shape: (7093, 300, 23), y_test shape: (7093,)
X_val shape: (7100, 300, 23), y_val shape: (7100,)


## Plot ROC Curve of best model for 10, 20 and 60 seconds data

In [ ]:
fig, ax = plt.subplots()

for experiment_id in EXPERIMENT_IDS:
    plot_roc_curve_from_log_dir(experiment_id=experiment_id, ax=ax, pos_label=1)

### Plot learning curve of best model for 10, 20 and 60 seconds data


In [ ]:
% load_ext autoreload
% autoreload 2

In [ ]:
plt.rcParams["figure.figsize"] = 10, 20

n_estimator_options = np.logspace(2, 9, num=10, base=2)
_, axes = plt.subplots(len(EXPERIMENT_IDS), 1)
for experiment_id, ax in zip(EXPERIMENT_IDS, axes):
    plot_learning_curve_from_log_dir(experiment_id=experiment_id,
                                     n_estimator_options=n_estimator_options, ax=ax)


### Plot classification report of best model for 10, 20 and 60 seconds data


In [ ]:
for experiment_id in EXPERIMENT_IDS:
    exp_config, best_estimator, _ = load_experiment_objects(experiment_id=experiment_id)

    window_size = exp_config["window_in_sec"]
    config.set_paths(30, window_size)

    # load data
    X_train, X_test, y_train, y_test = load_preprocessed_train_test_splits(
        data_path=config.PATHS.WINDOW_FEATURES,
        exclude_sess_type=session_type_mapping[exp_config["exclude_by"]],
        num_targets=exp_config["num_targets"],
        seed=exp_config["seed"],
        test_size=exp_config["test_size"])

    y_hat = best_estimator.predict(X_test)
    report = classification_report(y_true=y_test, y_pred=y_hat,
                                   target_names=label_names_dict[exp_config["num_targets"]])
    print(report)